# LOG8415E: Personal project demo

**Before running this notebook it is essential to configure aws credentials in
to be able to interact with AWS.**

**The shared credentials file has a default location of ~/.aws/credentials.
Put your information there.**

## Importing modules and deploying stantalone, cluster and proxy instances

In [ ]:
import client
import mysql_ec2_helper
import boto3
import os
import time
import nest_asyncio
from fabric import Connection

nest_asyncio.apply()

In [ ]:
mysql_ec2_helper.deploy_standalone()
mysql_ec2_helper.deploy_cluster()
mysql_ec2_helper.deploy_proxy()

## Standalone instance user data

## Slave instance user data

## Application instance user data

## Getting benchmark results from standalone instance and deploying gatekeeper

In [ ]:
def find_instance_ip(name_tag):
    client = boto3.client('ec2')

    response = client.describe_instances(
        Filters=[
            {
                'Name': 'tag:Name',
                'Values': [name_tag]
            }
        ]
    )

    ip = [i['PublicIpAddress'] for r in response['Reservations'] for i in r['Instances'] if i['State']['Name'] == 'running']
    
    return ip

In [ ]:
if not os.path.exists('results'):
    os.makedirs('results')
    
standalone_ip = find_instance_ip('standalone')[0]

with Connection(standalone_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/benchmark_standalone.txt', local='results/benchmark_standalone.txt')
    
mysql_ec2_helper.terminate_instance('standalone')

mysql_ec2_helper.deploy_gatekeeper()

## Launching client application for proxy server
1. Opens a tcp connection with proxy server on port 5001.
2. Reads data file 'people.csv'.
3. Sends 'INSERT' SQL requests to database via proxy server.
4. Sends 'SELECT' SQL requests to database via proxy server using 3 different modes: direct hit, random and custom
6. Saves response times for each mode in results folder.
5. Closes tcp connection with proxy server.

In [ ]:
client.main('proxy')

## Getting benchmark results from cluster instances as well as power consumption data

In [ ]:
master_ip = find_instance_ip('master')[0]
slave_1_ip = find_instance_ip('slave')[0]
slave_2_ip = find_instance_ip('slave')[1]
proxy_ip = find_instance_ip('proxy')[0]

with Connection(master_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/benchmark_replication.txt', local='results/benchmark_replication.txt')
    c.get('/tmp/powerapi_master.txt', local='results/powerapi_master_1.txt')

with Connection(slave_1_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_slave.txt', local='results/powerapi_slave1_1.txt')

with Connection(slave_2_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_slave.txt', local='results/powerapi_slave2_1.txt')

with Connection(proxy_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_proxy.txt', local='results/powerapi_proxy_1.txt')

## Changing security groups rules for trusted host
1. Create a new security group for gatekeeper instance only allowing incoming tcp connections on port 5002.
2. Change security group rules for trusted host (proxy instance in my case) as well as cluster instances.
3. Rebooting instances for changes to take effect.

In [ ]:
mysql_ec2_helper.adjust_security_group_rules_with_gatekeeper()

mysql_ec2_helper.reboot_instances(['master', 'slave'])
time.sleep(5)

mysql_ec2_helper.reboot_instances(['proxy'])
time.sleep(5)

mysql_ec2_helper.reboot_instances(['gatekeeper'])

## Launching client application for gatekeeper server
1. Opens a tcp connection with gatekeeper server on port 5002.
2. Gatekeeper establishes a tcp connection with a trusted host (proxy) on port 5001.
2. Reads data file 'people.csv'.
3. Sends 'INSERT' SQL requests to database via gatekeeper server.
4. Sends 'SELECT' SQL requests to database via gatekeeper server using 3 different modes for proxy: direct hit, random and custom
6. Saves response times for each mode in results folder.
5. Closes tcp connection with gatekeeper server.

In [ ]:
client.main('gatekeeper')

## Getting power consumption data from cluster instances as well as proxy and gatekeeper instances

In [ ]:
gatekeeper_ip = find_instance_ip('gatekeeper')[0]

print("Getting results from instances")
with Connection(master_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_master.txt', local='results/powerapi_master_2.txt')

with Connection(slave_1_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_slave.txt', local='results/powerapi_slave1_2.txt')

with Connection(slave_2_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_slave.txt', local='results/powerapi_slave2_2.txt')

with Connection(proxy_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_proxy.txt', local='results/powerapi_proxy_2.txt')

with Connection(gatekeeper_ip, user='ubuntu', connect_kwargs={'key_filename': 'mysql.pem'}) as c:
    c.get('/tmp/powerapi_gatekeeper.txt', local='results/powerapi_gatekeeper.txt')

## Stopping instances

In [ ]:
mysql_ec2_helper.stop_instances(['master', 'slave', 'proxy', 'gatekeeper']) # or mysql_ec2_helper.cleanup()